# Búsqueda de algoritmos para Alemania

In [1]:
# Load packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

/home/juancarlos/Desktop/personal/explanable-model-drift/.pyenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Carga de datos y preprocesamiento

In [2]:
dataset = pd.read_csv('/home/juancarlos/Desktop/personal/explanable-model-drift/results/preprocessing/tourism_alemania.csv', sep=";", decimal=".", encoding="utf-8")
date_column = 'Fecha'
target_column = 'Alemania'

dataset[date_column] = pd.to_datetime(dataset[date_column])
dataset.rename(columns={f"Llegadas a AGP {target_column}": 'Llegadas a AGP'}, inplace=True)

dataset.head()

,Alemania,Pib Pc,IPC Armonizado,Desempleo Armonizado,Asientos ofertados,IPC España Armonizado,Llegadas a AGP,Mortalidad,Indice del consumidor,Búsquedas hacia AGP total,Búsquedas hacia AGP 3 meses,Búsquedas hacia AGP 6 meses,Fecha
0,76517.0,6770.0,84.3,11.5,-1,81.16,23662.0,73561,-13.7,0,0,0,2005-01-01
1,89108.0,6770.0,84.6,11.5,-1,81.35,26658.0,72275,-14.5,0,0,0,2005-02-01
2,113595.0,6770.0,84.9,11.5,-1,82.07,43621.0,83271,-15.2,0,0,0,2005-03-01
3,135801.0,6980.0,84.9,11.3,-1,83.24,47549.0,68245,-15.4,0,0,0,2005-04-01
4,181280.0,6980.0,85.2,11.3,-1,83.37,64567.0,69328,-14.7,0,0,0,2005-05-01


In [3]:
dataset['Mes'] = dataset[date_column].dt.month
dataset = dataset.set_index(dataset[date_column])
dataset = dataset.drop(columns=[date_column])
dataset

,Alemania,Pib Pc,IPC Armonizado,Desempleo Armonizado,Asientos ofertados,IPC España Armonizado,Llegadas a AGP,Mortalidad,Indice del consumidor,Búsquedas hacia AGP total,Búsquedas hacia AGP 3 meses,Búsquedas hacia AGP 6 meses,Mes
Fecha,,,,,,,,,,,,,
2005-01-01,76517.0,6770.0,84.3,11.5,-1,81.16,23662.0,73561,-13.7,0,0,0,1
2005-02-01,89108.0,6770.0,84.6,11.5,-1,81.35,26658.0,72275,-14.5,0,0,0,2
2005-03-01,113595.0,6770.0,84.9,11.5,-1,82.07,43621.0,83271,-15.2,0,0,0,3
2005-04-01,135801.0,6980.0,84.9,11.3,-1,83.24,47549.0,68245,-15.4,0,0,0,4
2005-05-01,181280.0,6980.0,85.2,11.3,-1,83.37,64567.0,69328,-14.7,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-01,0.0,12250.0,126.6,3.0,-1,120.03,52854.0,0,-14.0,0,0,0,8
2024-09-01,0.0,12250.0,126.6,3.0,-1,120.03,52854.0,0,-14.0,0,0,0,9
2024-10-01,0.0,12250.0,126.6,3.0,-1,120.03,52854.0,0,-14.0,0,0,0,10


In [4]:
arrivals = dataset[[target_column,f'Llegadas a AGP','Pib Pc','Desempleo Armonizado',
                    'IPC Armonizado', 'Mes', 'Indice del consumidor', 'Mortalidad']] #'Asientos ofertados',
arrivals

,Alemania,Llegadas a AGP,Pib Pc,Desempleo Armonizado,IPC Armonizado,Mes,Indice del consumidor,Mortalidad
Fecha,,,,,,,,
2005-01-01,76517.0,23662.0,6770.0,11.5,84.3,1,-13.7,73561
2005-02-01,89108.0,26658.0,6770.0,11.5,84.6,2,-14.5,72275
2005-03-01,113595.0,43621.0,6770.0,11.5,84.9,3,-15.2,83271
2005-04-01,135801.0,47549.0,6980.0,11.3,84.9,4,-15.4,68245
2005-05-01,181280.0,64567.0,6980.0,11.3,85.2,5,-14.7,69328
...,...,...,...,...,...,...,...,...
2024-08-01,0.0,52854.0,12250.0,3.0,126.6,8,-14.0,0
2024-09-01,0.0,52854.0,12250.0,3.0,126.6,9,-14.0,0
2024-10-01,0.0,52854.0,12250.0,3.0,126.6,10,-14.0,0


In [5]:
# Crear características de lags
# for lag in range(1, 13):
#     arrivals[f'lag_{lag}'] = arrivals[target_column].shift(lag)

lag = 12
arrivals[f'lag_{lag}'] = arrivals[target_column].shift(lag)

# # # Crear características de ventanas móviles
for window in [12]:
    arrivals[f'rolling_mean_{window}'] = arrivals[target_column].rolling(window=window).mean()
    arrivals[f'rolling_std_{window}'] = arrivals[target_column].rolling(window=window).std()

# # Crear características de estacionalidad
# df['month'] = df.index.month
# df['quarter'] = df.index.quarter
# df['year'] = df.index.year

# Eliminar filas con valores NaN (resultantes de los lags y ventanas móviles)
arrivals = arrivals.dropna()

# # Dividir los datos en características (X) y etiquetas (y)
# X = df.drop(columns=['arrivals'])
# y = df['arrivals']
arrivals

/tmp/ipykernel_547041/2168137393.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arrivals[f'lag_{lag}'] = arrivals[target_column].shift(lag)
/tmp/ipykernel_547041/2168137393.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arrivals[f'rolling_mean_{window}'] = arrivals[target_column].rolling(window=window).mean()
/tmp/ipykernel_547041/2168137393.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

,Alemania,Llegadas a AGP,Pib Pc,Desempleo Armonizado,IPC Armonizado,Mes,Indice del consumidor,Mortalidad,lag_12,rolling_mean_12,rolling_std_12
Fecha,,,,,,,,,,,
2006-01-01,88568.0,25446.0,7080.0,11.3,86.1,1,-7.4,74230,76517.0,141508.416667,48610.777231
2006-02-01,104246.0,28086.0,7080.0,11.3,86.4,2,-8.2,66793,89108.0,142769.916667,47306.228434
2006-03-01,163785.0,43915.0,7080.0,11.3,86.5,3,-8.6,73680,113595.0,146952.416667,46707.221580
2006-04-01,171910.0,55077.0,7220.0,10.2,86.8,4,-8.3,68105,135801.0,149961.500000,47085.108539
2006-05-01,181345.0,56241.0,7220.0,10.2,87.0,5,-5.9,67837,181280.0,149966.916667,47089.042525
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-01,0.0,52854.0,12250.0,3.0,126.6,8,-14.0,0,128291.0,32229.666667,59891.138853
2024-09-01,0.0,52854.0,12250.0,3.0,126.6,9,-14.0,0,165641.0,18426.250000,43075.187391
2024-10-01,0.0,52854.0,12250.0,3.0,126.6,10,-14.0,0,114950.0,8847.083333,30647.195664


In [6]:
def split_by_datetime(dataframe, start_date, end_date):
    """
    Given a pandas dataset and start date/end date, select a portion of this dataset
    
    Date format: 'yyyy-mm-dd'
    
    Returns a portion of the original dataset
    """
    mask = (dataframe.index>= start_date) & (dataframe.index <= end_date)
    dataframe = dataframe.loc[mask]
    return dataframe

dataset_train = split_by_datetime(arrivals,'2009-01-01','2016-12-01')
dataset_test = split_by_datetime(arrivals,'2017-01-01','2017-12-01')
dataset_val = split_by_datetime(arrivals,'2018-01-01', dataset.index[-1])

In [7]:
y_train = np.array(dataset_train[target_column])
# Remove the labels from the features
X_train = dataset_train.drop(target_column, axis = 1)
# Saving feature names for later use
feature_list = list(dataset_train.columns)
print(X_train.shape)
print(y_train.shape)

# Labels are the values we want to predic
y_test = np.array(dataset_test[target_column])
# Remove the labels from the features
X_test = dataset_test.drop(target_column, axis = 1)
# Convert to numpy array
# X_test = np.array(dataset_test)
print(X_test.shape)
print(y_test.shape)

# Labels are the values we want to predict
y_val = np.array(dataset_val[target_column])
# Remove the labels from the features
X_val = dataset_val.drop(target_column, axis = 1)
# Convert to numpy array
# X_val = np.array(dataset_val)
print(X_val.shape)
print(y_val.shape)


(96, 10)
(96,)
(12, 10)
(12,)
(84, 10)
(84,)


## Grid Search

In [8]:
def grid_search(
        base_model,
        X_train: pd.DataFrame,
        y_train: pd.Series,
        X_test: pd.DataFrame,
        y_test: pd.Series,
        param_distributions: dict,
        test_size: int = 12,
        n_splits: int = 5,
        n_jobs: int = -1
):
    """
    Function to perform time series modeling with RandomForestRegressor and hyperparameter tuning.

    Parameters:
    base_model (object): Base model to use for hyperparameter tuning
    X_train (pd.DataFrame): Training features
    y_train (pd.Series): Training target
    X_test (pd.DataFrame): Test features
    y_test (pd.Series): Test target
    param_distributions (dict): Hyperparameter grid for GridSearchCV
    test_size (int): Size of the test set for TimeSeriesSplit
    n_splits (int): Number of splits for TimeSeriesSplit
    n_jobs (int): Number of jobs to run in parallel for GridSearchCV

    Returns:
    pd.DataFrame: Cross-validation results
    dict: Best hyperparameters found
    dict: Test set evaluation metrics
    """
    
    # Create TimeSeriesSplit object
    tscv = TimeSeriesSplit(n_splits=n_splits, test_size=test_size)

    # Hyperparameter search with GridSearchCV
    gd_search = GridSearchCV(
        estimator=base_model,
        param_grid=param_distributions,
        cv=tscv,
        n_jobs=n_jobs
    )

    # Train the hyperparameter search
    gd_search.fit(X_train, y_train)

    # Best model found
    gd_best_model = gd_search.best_estimator_

    # Print best model score
    print(f"Best model score: {gd_best_model.score(X_train, y_train)}")

    # Cross-validation results
    results = []
    for train_index, val_index in tscv.split(X_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
        print(f"Validation start date: {X_val_fold.index[0]}")
        
        # Train the model with the best parameters
        gd_best_model.fit(X_train_fold, y_train_fold)

        # Make predictions
        y_val_pred = gd_best_model.predict(X_val_fold)

        # Calculate metrics
        mse = mean_squared_error(y_val_fold, y_val_pred)
        r2 = r2_score(y_val_fold, y_val_pred)
        mae = mean_absolute_error(y_val_fold, y_val_pred)

        # Store results
        results.append({'MSE': mse, 'R2': r2, 'MAE': mae})

    # Convert results to DataFrame for analysis
    results_df = pd.DataFrame(results)
    print("Cross-validation results:")
    print(results_df)

    # Final evaluation on test set
    gd_best_model.fit(X_train, y_train)
    y_test_pred = gd_best_model.predict(X_test)

    # Calculate test set metrics
    test_mse = mean_squared_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)

    test_metrics = {
        'Test MSE': test_mse,
        'Test R2': test_r2,
        'Test MAE': test_mae
    }
    
    print(f"Test MSE: {test_mse}")
    print(f"Test R2: {test_r2}")
    print(f"Test MAE: {test_mae}")
    print(f"Best hyperparameters: {gd_search.best_params_}")

    return results_df, gd_best_model, gd_search.best_params_, test_metrics

In [12]:
# Define hyperparameter grids
random_state = 10
param_grids = {
    'RandomForestRegressor': {
        'n_estimators': [10, 50, 100, 200, 300, 400, 600, 1000],
        'max_features': [X_test.shape[1], 'sqrt', 'log2'],
        'max_depth': [10, 20, 30, 40, 50, None],
        'min_samples_split': [1, 2, 4],
        'min_samples_leaf': [2, 5, 10],
        'random_state': [random_state]
    },
    'LinearRegression': {
        'fit_intercept': [True, False],
    },
    'GradientBoostingRegressor': {
        'n_estimators': [10, 50, 100, 200, 300, 400, 600, 1000],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'subsample': [0.6, 0.8, 1.0],
        'random_state': [random_state]
    },
    'MLPRegressor': {
        'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
        'activation': ['relu', 'tanh', 'logistic'],
        'solver': ['adam', 'sgd'],
        'alpha': [0.0001, 0.001, 0.01],
        'learning_rate': ['constant', 'invscaling', 'adaptive'],
        'max_iter': [200, 500, 1000],
        'random_state': [random_state]
    }
}

# Define models
models = {
    'RandomForestRegressor': RandomForestRegressor(),
    'LinearRegression': LinearRegression(),
    'GradientBoostingRegressor': GradientBoostingRegressor(),
    'MLPRegressor': MLPRegressor()
}

# Perform grid search for each model and collect results
results = {}
for model_name, model in models.items():
    print(f"Training {model_name}...")
    param_grid = param_grids[model_name]
    results[model_name] = grid_search(
        base_model=model,
        X_train=X_train,
        y_train=pd.Series(y_train, index=dataset_train.index),
        X_test=X_test,
        y_test=pd.Series(y_test, index=dataset_test.index),
        param_distributions=param_grid
    )

Training RandomForestRegressor...


/home/juancarlos/Desktop/personal/explanable-model-drift/.pyenv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
2160 fits failed out of a total of 6480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2160 fits failed with the following error:
Traceback (most recent call last):
  File "/home/juancarlos/Desktop/personal/explanable-model-drift/.pyenv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/juancarlos/Desktop/personal/explanable-model-drift/.pyenv/lib/python3.10/site-packages/sklearn/base.py", line 1467, in wrapper
    estimator._validate_params()
  File "/home/juancarlos/Deskt

Best model score: 0.9832776559560088
Validation start date: 2012-01-01 00:00:00
Validation start date: 2013-01-01 00:00:00
Validation start date: 2014-01-01 00:00:00
Validation start date: 2015-01-01 00:00:00
Validation start date: 2016-01-01 00:00:00
Cross-validation results:
            MSE        R2           MAE
0  2.450884e+08  0.892367  13842.880384
1  1.069367e+09  0.638618  23641.550720
2  3.184410e+08  0.894521  14467.770683
3  1.810946e+08  0.928433  11973.872699
4  9.616290e+07  0.966037   7937.483088
Test MSE: 605566509.733839
Test R2: 0.8200279603571003
Test MAE: 19659.54658945106
Best hyperparameters: {'max_depth': 20, 'max_features': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 600, 'random_state': 10}
Training LinearRegression...
Best model score: 0.9136345731377922
Validation start date: 2012-01-01 00:00:00
Validation start date: 2013-01-01 00:00:00
Validation start date: 2014-01-01 00:00:00
Validation start date: 2015-01-01 00:00:00
Validation st

/home/juancarlos/Desktop/personal/explanable-model-drift/.pyenv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/juancarlos/Desktop/personal/explanable-model-drift/.pyenv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/juancarlos/Desktop/personal/explanable-model-drift/.pyenv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/juancarlos/Desktop/personal/explanable-model-drift/.pyenv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: Converg

Best model score: 0.8967541645966791
Validation start date: 2012-01-01 00:00:00
Validation start date: 2013-01-01 00:00:00


/home/juancarlos/Desktop/personal/explanable-model-drift/.pyenv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/juancarlos/Desktop/personal/explanable-model-drift/.pyenv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Validation start date: 2014-01-01 00:00:00
Validation start date: 2015-01-01 00:00:00


/home/juancarlos/Desktop/personal/explanable-model-drift/.pyenv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/juancarlos/Desktop/personal/explanable-model-drift/.pyenv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Validation start date: 2016-01-01 00:00:00
Cross-validation results:
            MSE        R2           MAE
0  1.819442e+08  0.920098  10889.186517
1  4.869713e+08  0.835433  17724.072127
2  4.014753e+08  0.867017  14040.142628
3  1.221307e+08  0.951735   9508.460649
4  1.568958e+08  0.944587   9734.936055
Test MSE: 386843089.51814055
Test R2: 0.8850317203424267
Test MAE: 14620.237666337454
Best hyperparameters: {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (50,), 'learning_rate': 'constant', 'max_iter': 500, 'random_state': 10, 'solver': 'adam'}


/home/juancarlos/Desktop/personal/explanable-model-drift/.pyenv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/juancarlos/Desktop/personal/explanable-model-drift/.pyenv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [32]:
models_results = pd.DataFrame()
models_tests = pd.DataFrame()
for model_name, (cv_results, best_model, best_params, test_metrics) in results.items():
    cv_results['Model'] = model_name
    models_results = pd.concat([models_results, cv_results])
    test_metrics['Model'] = model_name
    models_tests = pd.concat([models_tests, pd.DataFrame(test_metrics, index=[0])])

models_results = models_results.groupby('Model').mean()

print("Results")
print(models_results)

print("R2 max:", models_results[models_results['R2'] == models_results['R2'].max()].index.values[0])
print("MSE min:", models_results[models_results['MSE'] == models_results['MSE'].min()].index.values[0])
print("MAE min:", models_results[models_results['MAE'] == models_results['MAE'].min()].index.values[0])
print()
print("Tests")
print(models_tests)

print("R2 max:", models_tests[models_tests['Test R2'] == models_tests['Test R2'].max()]["Model"])
print("MSE min:", models_tests[models_tests['Test MSE'] == models_tests['Test MSE'].min()]["Model"])
print("MAE min:", models_tests[models_tests['Test MAE'] == models_tests['Test MAE'].min()]["Model"])


Results
                                    MSE        R2           MAE
Model                                                          
GradientBoostingRegressor  2.623331e+08  0.907959  12028.126030
LinearRegression           5.595849e+08  0.796576  18849.316725
MLPRegressor               2.698835e+08  0.903774  12379.359595
RandomForestRegressor      3.820309e+08  0.863995  14372.711515
R2 max: GradientBoostingRegressor
MSE min: GradientBoostingRegressor
MAE min: GradientBoostingRegressor

Tests
       Test MSE   Test R2      Test MAE                      Model
0  6.055665e+08  0.820028  19659.546589      RandomForestRegressor
0  4.135643e+08  0.877090  16469.325324           LinearRegression
0  7.024409e+08  0.791237  20654.623067  GradientBoostingRegressor
0  3.868431e+08  0.885032  14620.237666               MLPRegressor
R2 max: 0    MLPRegressor
Name: Model, dtype: object
MSE min: 0    MLPRegressor
Name: Model, dtype: object
MAE min: 0    MLPRegressor
Name: Model, dtype: object
